# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '../../../../notebooks/' not in sys.path:
    sys.path.append('../../../../notebooks/')
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from base.model_evaluator import DataSource, DataPredSelection
from base.base_models import BaseModel
from base.optimizers import Optimizer
from base.model_creator import MTLApproach, NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [3]:
kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'neural_arch_search',
        'description': 'Making NAS with APPROACH_1 (random) and training best architecture for 50 epochs to evaluate on test set',
        'tags': ['ground truths', 'nas', 'nas_approach_1'],
        'src_files': ["src/**/*.py"]
    },
    'properties': {
        'approach': NAS_MTLApproach.APPROACH_1,
        'reqs': list(cts.ICAO_REQ),
        'aligned': False,
        'use_gt_data': True,
        'gt_names': {
            'train_validation': [],
            'test': [],
            'train_validation_test': [GTName.FVC]
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 32,
        'n_epochs': 50,
        'early_stopping': 99,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {
        'max_blocks_per_branch': 5,
        'n_epochs': 5,
        'n_trials': 30
    }
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 71660
Process ID: 85591
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'exp_params': {'description': 'Making NAS with APPROACH_1 (random) and '
                               'training best architecture for 50 epochs to '
                               'evaluate on test set',
                'name': 'neural_arch_search',
                'src_files': ['src/**/*.py'],
                'tags': ['ground truths', 'nas', 'nas_approach_1']},
 'nas_params': {'max_blocks_per_branch': 5, 'n_epochs': 5, 'n_trials': 30},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f99cebff430>}>,
                      'batch_size': 32,
                      'dropout': 0.3,
                      'early_stopping': 99,
                      'learning_rate': 0.001,
                    

# Load Data

In [4]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4928, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


# Data Generators

In [5]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 4928 validated image filenames.
Found 547 validated image filenames.
Found 288 validated image filenames.
TOTAL: 5763

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Setup Experiment

In [6]:
runner.setup_experiment()

-------------------- create experiment -------------------
Setup neptune properties and parameters
Properties and parameters setup done!


# Labels Distribution

In [7]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search

In [ ]:
runner.run_neural_architeture_search()

-------------------- run neural architecture search -------------------
++++++++++++++++++++ STARTING NEW TRAIN ++++++++++++++++++++
 ----- Training 1 | Config: {'n_denses_0': 3, 'n_denses_1': 2, 'n_denses_2': 2, 'n_denses_3': 1} --------
Creating model...
Model created
Training VGG16 network
 .. Not fine tuning base model...
  .. Total params: 15,572,654
  .. Trainable params: 857,966
  .. Non-trainable params: 14,714,688
Epoch 1/5


154/154 [==============================] - 417s 3s/step - loss: 1.0839 - background_loss: 0.6357 - close_loss: 0.5601 - ink_mark_loss: 0.6557 - pixelation_loss: 0.5328 - washed_out_loss: 0.0943 - blurred_loss: 0.4172 - sh_head_loss: 0.4000 - mouth_loss: 0.0282 - veil_loss: 0.6546 - red_eyes_loss: 0.5495 - flash_lenses_loss: 0.3074 - dark_glasses_loss: 0.7277 - l_away_loss: 0.3605 - frame_eyes_loss: 0.7259 - hair_eyes_loss: 0.6667 - eyes_closed_loss: 0.3920 - frames_heavy_loss: 0.0207 - sh_face_loss: 0.8459 - skin_tone_loss: 0.1235 - light_loss: 0.2926 - hat_loss: 0.5251 - rotation_loss: 0.7576 - reflection_loss: 0.5655 - background_accuracy: 0.7190 - close_accuracy: 0.7524 - ink_mark_accuracy: 0.6715 - pixelation_accuracy: 0.7681 - washed_out_accuracy: 0.9880 - blurred_accuracy: 0.8584 - sh_head_accuracy: 0.8711 - mouth_accuracy: 0.9972 - veil_accuracy: 0.6863 - red_eyes_accuracy: 0.7792 - flash_lenses_accuracy: 0.9200 - dark_glasses_accuracy: 0.5110 - l_away_accuracy: 0.9056 - frame_e

# Create Model

In [ ]:
runner.create_model()

# Vizualize Model

In [ ]:
runner.visualize_model(outfile_path=f"figs/nas/nas_model_approach_1.png")

In [ ]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model()

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model()

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()

# Testing Network Modification